In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
page = requests.get('https://www.avito.ma/fr/maroc/v%C3%A9hicules-%C3%A0_vendre')
soup = BeautifulSoup(page.text, 'html.parser')
l=[]
s=[]
#cherher nolbre de pages d'annonces de voitures
h=soup.find_all(class_='pagination text-center') 
l=h[0].find_all_next('a')
s=[ i['href'] for i in l if "https://www.avito.ma/fr/maroc/véhicules-à_vendre?" in  i['href'] and "class" not in str(i) ]
nbr_page_vehi=s[-1].replace('https://www.avito.ma/fr/maroc/véhicules-à_vendre?o=','')
print(nbr_page_vehi)#le voilà 
ty_pe=[]
ville=[]
secteur=[]
kilometrage=[]
modele=[]
marque=[]
carburant=[]
pui_fiscale=[]
boite_a_vitesse=[]
num_tel=[]
année_modele=[]
price=[]
location=[]
vues=[]
date_pub=[]
type_vendeur=[]
for i in range(250):
    try :  
        page_veh='https://www.avito.ma/fr/maroc/voitures-%C3%A0_vendre?o='+str(i)
        r=requests.get(page_veh)
        soup_it=BeautifulSoup(r.text,'html.parser')
        it=soup_it.find_all('h2',class_="fs14")
        item=[]
        #chercher les liens de chaque voiture dans une page
        for j in range(len(it)):
            item.append(it[j].a['href'])
        for link in item:
            r=requests.get(link)
            soup=BeautifulSoup(r.text,'html.parser')
            if "price-header" in str(soup):
                price.append(soup.find_all('h2',class_="price-header")[0].span.contents[0])
            else:
                price.append('?')
            info=soup.find_all('h2',class_="font-normal fs12 no-margin ln22")
            l=[année_modele,kilometrage,carburant,marque,modele,pui_fiscale,secteur,ty_pe]
            if not info:
                continue
            for k in [0,2,3,4,5,6,7]:
                if info[k].a!=None:
                    l[k].append(info[k].a.contents[0])
                else:
                    l[k].append('vide')
            l[1].append(info[1].contents[1].replace('\n',''))   
            loc=soup.find_all('h2',class_="font-normal fs13 lh30 no-margin")
            if loc[0]!=None:
                location.append(loc[0].contents[1])
            else:
                location.append('vide')
            v=soup.find_all('div',class_='span20')
            vues.append(v[0].span.contents[2].replace('total\n',''))
            l=soup.find_all('script')
            for i in l:
                if 'telephone' in str(i):
                    break
            para=i
            m=str(para)
            if 'telephone:' in m:
                s=m.index('telephone:')
                i=s+11
                num=''.join([m[i+k] for k in range(1,11)])
                num_tel.append(num)
            else:
                num_tel.append('')

            it=soup.find_all('div',class_="panel-body")   
            c=str(it[0].find_all('div')[0].find_all(class_='span10')[0])
            if 'Boite' in c:
                indi=c.index('Boite')
                t=''
                nom=c[indi+len('boite a vitesse')+3]
                if nom=='A':
                    t='automatique'
                else:
                    t='manuelle'
                boite_a_vitesse.append(t)
            else:
                boite_a_vitesse.append('')
            date_pub.append(soup.find_all('abbr',class_="date dtstart value")[0]['title'])
            vi=soup.find_all('h2' ,class_="font-normal fs13 lh30 no-margin")
            ville.append(vi[0].contents[1])
            pr=soup.find_all('h1',class_="page-header mbm")
            if pr[0].small!=None:
                type_vendeur.append(pr[0].small.contents[0])
            else:
                 type_vendeur.append('')
    except (ConnectionError,OSError):
        pass
        

4296


In [3]:
prix2=price[:len(price)-1]
len(prix2)
data={'ville':ville,'secteur':secteur,'marque':marque,'modèle':modele,'année modèle':année_modele,'carburant': carburant,'boite à vitesse':boite_a_vitesse,'puissance fiscale':pui_fiscale,'kilométrage':kilometrage,'date de publication':date_pub,'nbre de vues':vues,'prix':prix2,'telephone du vendeur':num_tel,'type annonce':type_vendeur}
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn
seaborn.set()
%matplotlib inline
df=pd.DataFrame(data)#création data frame
df.to_csv('scraping3.csv',encoding='utf-8', index=False) # création csv file

In [5]:
df

,ville,secteur,marque,modèle,année modèle,carburant,boite à vitesse,puissance fiscale,kilométrage,date de publication,nbre de vues,prix,telephone du vendeur,type annonce
0,Fès,vide,Volkswagen,GOLF 5,2008,Diesel,,8 CV,180 000 - 189 999,2020-04-04T06:59:34,3,?,0639908681,
1,Fès,vide,Hyundai,Santa Fe,2007,Diesel,,8 CV,200 000 - 249 999,2020-04-04T06:50:07,26,?,0639908681,
2,Ouarzazate,vide,Renault,Clio,2017,Diesel,,6 CV,55 000 - 59 999,2020-04-04T06:26:12,193,?,0652314897,
3,Kénitra,Autre secteur,Hyundai,i 30,2011,Diesel,manuelle,16 CV,70 000 - 74 999,2020-04-04T06:18:55,50,?,0645426170,
4,Nador,vide,Peugeot,208,2014,Diesel,,6 CV,85 000 - 89 999,2020-04-04T06:11:46,81,80 000,0653215436,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8709,Azrou,vide,Citroen,C15,2000,Diesel,manuelle,6 CV,180 000 - 189 999,2020-03-14T19:49:18,684,14 500,0666338862,
8710,El Jadida,vide,Dacia,Dokker,2014,Diesel,,6 CV,140 000 - 149 999,2020-03-14T19:49:15,1137,33 000,0663201850,
8711,Casablanca,vide,Citroen,C15,1999,Diesel,manuelle,7 CV,0 - 4 999,2020-03-14T19:49:09,858,73 000,0613249321,
8712,Fès,vide,Citroen,DS5,2019,Diesel,automatique,8 CV,10 000 - 14 999,2020-03-14T19:48:08,952,30 000,0620202095,
